In [1]:
import chess

from chess_app.environment import ChessInterface
from re_algoritms.agents import RandomAgent, QAgent
from chess_app.dataset.loader import loaderChessPos
import chess.svg
from tqdm import tqdm


In [2]:

def is_checkmate(board, state):
    board.set_fen(state)
    return board.is_checkmate()


In [ ]:
from re_algoritms.dqn import GrandMasterNet
from re_algoritms.agents import DQNAgent

board = chess.Board()

e = 0.1
lr = 0.3
gamma = 0.3
n_epochs = 100

policy_net = GrandMasterNet(
    board_vec_dim=64,
    moves_vec_dim=5
)
player = DQNAgent(
    lr=lr, gamma=gamma, model=policy_net
)
loader = loaderChessPos(r'./chess_app/dataset')

In [ ]:
# bc_win = 0
# wh_win = 0
# stockfish = ChessInterface(verbose=True, engine_path=r"./chess_app/src/stockfish_15_x64_avx2.exe")
# for epoch in range(n_epochs):
#     for cnt_to_finish in list(loader.games.keys()):
#         for state in tqdm(loader.games[cnt_to_finish]):
#             stockfish.stockfish.set_fen_position(state)
#             # actions = list() # todo: выбранная вероятность
#             rewards = list()     # ...
#
#             move_counter = 1
#             reward = 0
#             black_win = False
#             white_win = False
#             new_state = state
#             b_state = None
#             while not is_checkmate(board, new_state):
#                 stockfish.stockfish.set_fen_position(new_state)
#                 player_move = player.return_move(new_state, stockfish, e)
#                 #print(f'Player move:{player_move}')
#                 stockfish.player_move(player_move)
#                 #new_state = stockfish.stockfish.get_fen_position()
#             # actions.append(new_state) # ?
#                 move_counter += 1
#                 if move_counter > 5:
#                     black_win = True
#                     bc_win += 1
#                     break
#                 # actions.append(new_state)
#
#                 if is_checkmate(board, new_state):
#                     wh_win += 1
#                     white_win = True
#                 else:
#                     machine_move = stockfish.env_move()
#                     new_state = stockfish.get_board_fen()
#                     #print(f'Machine move:{machine_move}')
#                 if is_checkmate(board, new_state) and not white_win:
#                     bc_win += 1
#                     black_win = True
#                 print(actions)
#
#
#
#                 #print('---')
#                 # break
#
#             if black_win is True:
#                 reward = -1
#             else:
#                 reward = 1
#
#             player.update_policy(reward=reward)
#             # break
#         print(f'Win: {wh_win}\t Lose: {bc_win}')

In [ ]:
bc_win = 0
wh_win = 0
stockfish = ChessInterface(verbose=True, engine_path=r"./chess_app/src/stockfish_15_x64_avx2.exe")
for epoch in range(n_epochs):
    for cnt_to_finish in list(loader.games.keys()):
        for state in tqdm(loader.games[cnt_to_finish]):
            new_state = state
            # init necessary for game

            black_win = False
            white_win = False

            stockfish.set_board_fen(state)
            all_rewards = list()

            for _ in range(5):
                # player make move
                stockfish.set_board_fen(new_state)
                player_move = player.return_move(
                    state,
                    stockfish.get_top_steps() # todo
                )
                stockfish.player_move(player_move)

                all_rewards.append(0)

                if is_checkmate(board, new_state):
                    white_win = True
                    break

                # environment make move
                machine_move = stockfish.env_move()
                new_state = stockfish.get_board_fen()

                if is_checkmate(board, new_state):
                    black_win = True
                    break
            else:
                black_win = True


            if black_win is True:
                reward = -1
            else:
                reward = 1

            player.update_policy(reward=all_rewards)